In [ ]:
import numpy as np
import awkward
import uproot

import matplotlib

fig_width = 5
fig_height = 4
params = {
          #'backend': 'notebook',
          'text.latex.preamble': [r'\usepackage{gensymb}'],
          'axes.labelsize': 8, # fontsize for x and y labels (was 10)
          'axes.titlesize': 8,
          'font.size': 8, # was 10
          'text.usetex': False,
          'figure.figsize': [fig_width,fig_height],
          'font.family': 'serif'
}

matplotlib.rcParams.update(params)
    
import matplotlib.pyplot as plt
import itertools
import numpy as np
import numba

In [ ]:
fi = uproot.open("../data/nanoaod_test.root")
tt = fi.get("Events")

In [ ]:
pts = tt.array("Jet_pt").content
etas = tt.array("Jet_eta").content
phis = tt.array("Jet_phi").content

offsets = tt.array("Jet_pt").offsets

In [ ]:
pt_jag = awkward.JaggedArray.fromoffsets(np.array(offsets, dtype=np.int64), pts)
eta_jag = awkward.JaggedArray.fromoffsets(offsets, etas)
phi_jag = awkward.JaggedArray.fromoffsets(offsets, phis)

In [ ]:
def fill_matrix(jag, Nx, Ny):
    mat = np.zeros((Nx,Ny), dtype=np.float32)
    for i in range(mat.shape[0]):
        for j in range(min(jag.counts[i], mat.shape[1])):
            mat[i,j] = jag[i][j]
    return mat

In [ ]:
Nev = 50
Njet = 20
mat_pt = fill_matrix(pt_jag, Nev, Njet)
mat_eta = fill_matrix(eta_jag, Nev, Njet)
mat_phi = fill_matrix(phi_jag, Nev, Njet)

In [ ]:
plt.figure(figsize=(6,4))
plt.suptitle("Jagged event content: {0} events, up to {1} jets".format(Nev, Njet), y=1.01, fontsize=14)

plt.subplot(1,4,1)
#plt.imshow(pt_jag.counts[:Nev].reshape(Nev,1), aspect=100.0 / Nev / Njet / 2, cmap="YlOrRd", interpolation="nearest")
plt.pcolormesh(pt_jag.counts[:Nev].reshape(Nev,1), cmap="Reds")
plt.gca().invert_yaxis()
plt.title("$N_{jets}$")
plt.ylabel("Event index")
plt.xticks([])
plt.colorbar()

plt.subplot(1,4,2)
#plt.imshow(mat_pt, cmap="YlOrRd", aspect="equal", interpolation="nearest")
plt.pcolormesh(mat_pt, cmap="YlOrRd")
plt.gca().invert_yaxis()
plt.xlabel("jet index")
plt.xticks([])
plt.yticks([])
plt.title("$p_T$")
plt.colorbar()

plt.subplot(1,4,3)
plt.pcolormesh(mat_eta, cmap="RdYlBu_r")
plt.gca().invert_yaxis()
plt.xticks([])
plt.yticks([])
plt.xlabel("jet index")
plt.title("$\eta$")
plt.colorbar()

plt.subplot(1,4,4)
#plt.imshow(mat_phi, cmap="RdYlBu_r", aspect="equal", interpolation="nearest")
plt.pcolormesh(mat_phi, cmap="RdYlBu_r")
plt.gca().invert_yaxis()
plt.xticks([])
plt.yticks([])
plt.xlabel("jet index")
plt.title("$\phi$")
plt.colorbar()

plt.tight_layout()
plt.savefig("../paper/plots/jagged_matrices.pdf", bbox_inches="tight")

In [ ]:
pt = pt_jag

In [ ]:
def sumpt_event(data, offsets, ret):
    ret[:] = 0
    for iev in range(len(offsets)-1):
        i0 = offsets[iev]
        i1 = offsets[iev+1]
        for j in range(i0, i1):
            ret[iev] += data[j]
    return ret

sum1 = np.zeros(len(offsets)-1, dtype=np.float64)
%timeit sumpt_event(pt_jag.content, pt_jag.offsets, sum1)

In [ ]:
%timeit -n3 pt_jag.sum()

In [ ]:
@numba.njit(parallel=True)
def sumpt_event(data, offsets, ret):
    ret[:] = 0
    for iev in range(len(offsets)-1):
        i0 = offsets[iev]
        i1 = offsets[iev+1]
        for j in range(i0, i1):
            ret[iev] += data[j]
    return ret

sum1 = np.zeros(len(offsets)-1, dtype=np.float64)
%timeit -n3 sumpt_event(pt_jag.content, pt_jag.offsets, sum1)